In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

/home/ray/miniconda3/envs/pydev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# X(images), Y(labels)
X = tf.placeholder(tf.float32, [None, 28*28]) # inf. x 784(28*28)
Y = tf.placeholder(tf.int64, [None]) # inf. x 1

In [4]:
# CNN Model
def model(input_X):
    x_reshape = tf.reshape(X, [-1, 28, 28, 1]) #batch_size x width x height x channel

    # Conv 레이어1
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 20], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv1 = tf.Variable(tf.zeros([20]))
    h_conv1 = tf.nn.conv2d(x_reshape, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1 # batch_sizex28x28x20
    fmap_conv1 = tf.nn.tanh(h_conv1) # -> Feature(Activation) Map 생성

    # Pooling(Max) 레이어1
    # k_size = [one_image, width, hegiht, one_channel]
    h_pool1 = tf.nn.max_pool(fmap_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # batch_sizex14x14x20

    # Conv 레이어2
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 20, 50], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv2 = tf.Variable(tf.zeros([50]))
    h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2 # batch_sizex14x14x50
    fmap_conv2 = tf.nn.tanh(h_conv2) # -> Feature(Activation) Map 생성

    # Pooling(Max) 레이어2
    h_pool2 = tf.nn.max_pool(fmap_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # batch_sizex7x7x50

    # fully-connected 레이어 1
    W_fc1 = tf.Variable(tf.truncated_normal([50 * 7 * 7, 500], stddev=0.1))
    b_fc1 = tf.Variable(tf.zeros([500]))
    h_pool2_flat = tf.reshape(h_pool2, [-1, 50 * 7 * 7]) # batch_sizex(7x7x50)
    h_fc1 = tf.nn.tanh(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) # batch_sizex500

    # 출력(fully connected) 레이어 2 (10개 출력)
    class_num = 10
    W_fc2 = tf.Variable(tf.truncated_normal([500, class_num], stddev=0.1))
    b_fc2 = tf.Variable(tf.zeros([class_num]))
    pred = tf.matmul(h_fc1, W_fc2) + b_fc2 #batch_sizex10

    return pred

pred = model(X)

In [5]:
# training 반복 횟수
max_step = 10000

# Cost Function 설계 (Cross Entropy)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=pred)
cost = tf.reduce_mean(cross_entropy)

# Gradient descent Optimizer
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [6]:
# 학습 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(1, max_step + 1):
    X_images, X_labels = mnist.train.next_batch(batch_size=100)
    X_labels = np.where(X_labels)[1]
        
    sess.run(optimizer, feed_dict={X: X_images, Y: X_labels})

    # 로그
    loss = sess.run(cost, feed_dict={X: X_images, Y: X_labels})
    if epoch % 500 == 0 or epoch == 0:
        print("epoch: %05d,"%(epoch), "loss:", loss)

print("Finished! (loss : " + str(loss) + ")")

epoch: 00500, loss: 0.280954
epoch: 01000, loss: 0.092200086
epoch: 01500, loss: 0.20317645
epoch: 02000, loss: 0.11976582
epoch: 02500, loss: 0.11813197
epoch: 03000, loss: 0.06038809
epoch: 03500, loss: 0.10010008
epoch: 04000, loss: 0.09483671
epoch: 04500, loss: 0.085210055
epoch: 05000, loss: 0.0450934
epoch: 05500, loss: 0.058854304
epoch: 06000, loss: 0.03711055
epoch: 06500, loss: 0.059077427
epoch: 07000, loss: 0.07380367
epoch: 07500, loss: 0.10163574
epoch: 08000, loss: 0.022388654
epoch: 08500, loss: 0.03709133
epoch: 09000, loss: 0.0152575895
epoch: 09500, loss: 0.045772348
epoch: 10000, loss: 0.058884352
Finished! (loss : 0.058884352)


In [7]:
test_images, test_labels = mnist.test.next_batch(16)
test_labels = np.where(test_labels)[1]

batch_size = 100
nCnt = 0
tmp_sum = []
for i in range(5):
    x_test, test_label = mnist.test.next_batch(batch_size)
    arr_data = sess.run(pred, feed_dict={X: x_test})

    pred_val = tf.argmax(arr_data, 1)
    real_val = np.where(test_label)[1]
    
    for ib in range(batch_size):
        tmp_pred = pred_val.eval(session=sess)
        tmp_real = real_val
        
        if tmp_pred[ib] == tmp_real[ib]:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => True")
            nCnt += 1
        else:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => False")
        tmp_sum.append(nCnt)
        nCnt = 0
tmp_sum = np.array(tmp_sum)
print("Accuracy:", (tmp_sum.mean()))

예측값: 4 / 실제값: 4 => True
예측값: 0 / 실제값: 0 => True
예측값: 1 / 실제값: 1 => True
예측값: 3 / 실제값: 3 => True
예측값: 4 / 실제값: 4 => True
예측값: 7 / 실제값: 7 => True
예측값: 9 / 실제값: 9 => True
예측값: 9 / 실제값: 9 => True
예측값: 0 / 실제값: 0 => True
예측값: 0 / 실제값: 0 => True
예측값: 5 / 실제값: 5 => True
예측값: 0 / 실제값: 0 => True
예측값: 1 / 실제값: 1 => True
예측값: 1 / 실제값: 1 => True
예측값: 5 / 실제값: 5 => True
예측값: 1 / 실제값: 1 => True
예측값: 6 / 실제값: 6 => True
예측값: 2 / 실제값: 2 => True
예측값: 9 / 실제값: 9 => True
예측값: 0 / 실제값: 0 => True
예측값: 2 / 실제값: 2 => True
예측값: 1 / 실제값: 1 => True
예측값: 2 / 실제값: 2 => True
예측값: 8 / 실제값: 8 => True
예측값: 7 / 실제값: 7 => True
예측값: 3 / 실제값: 3 => True
예측값: 2 / 실제값: 2 => True
예측값: 6 / 실제값: 6 => True
예측값: 3 / 실제값: 3 => True
예측값: 6 / 실제값: 1 => False
예측값: 3 / 실제값: 3 => True
예측값: 6 / 실제값: 6 => True
예측값: 7 / 실제값: 7 => True
예측값: 9 / 실제값: 9 => True
예측값: 5 / 실제값: 5 => True
예측값: 4 / 실제값: 4 => True
예측값: 3 / 실제값: 7 => False
예측값: 2 / 실제값: 2 => True
예측값: 5 / 실제값: 5 => True
예측값: 1 / 실제값: 1 => True
예측값: 2 / 실제값: 2 => True
예측값: 7 / 실제값: 

예측값: 6 / 실제값: 6 => True
예측값: 1 / 실제값: 1 => True
예측값: 9 / 실제값: 9 => True
예측값: 7 / 실제값: 7 => True
예측값: 4 / 실제값: 4 => True
예측값: 3 / 실제값: 3 => True
예측값: 3 / 실제값: 3 => True
예측값: 0 / 실제값: 0 => True
예측값: 6 / 실제값: 6 => True
예측값: 2 / 실제값: 2 => True
예측값: 6 / 실제값: 6 => True
예측값: 1 / 실제값: 1 => True
예측값: 4 / 실제값: 4 => True
예측값: 2 / 실제값: 2 => True
예측값: 6 / 실제값: 6 => True
예측값: 1 / 실제값: 1 => True
예측값: 9 / 실제값: 9 => True
예측값: 8 / 실제값: 8 => True
예측값: 1 / 실제값: 1 => True
예측값: 0 / 실제값: 0 => True
예측값: 4 / 실제값: 4 => True
예측값: 0 / 실제값: 0 => True
예측값: 0 / 실제값: 9 => False
예측값: 8 / 실제값: 8 => True
예측값: 6 / 실제값: 6 => True
예측값: 0 / 실제값: 0 => True
예측값: 5 / 실제값: 5 => True
예측값: 7 / 실제값: 7 => True
예측값: 1 / 실제값: 1 => True
예측값: 1 / 실제값: 1 => True
예측값: 9 / 실제값: 7 => False
예측값: 0 / 실제값: 0 => True
예측값: 2 / 실제값: 2 => True
예측값: 1 / 실제값: 1 => True
예측값: 5 / 실제값: 5 => True
예측값: 4 / 실제값: 4 => True
예측값: 8 / 실제값: 8 => True
예측값: 6 / 실제값: 6 => True
예측값: 8 / 실제값: 8 => True
예측값: 3 / 실제값: 3 => True
예측값: 9 / 실제값: 9 => True
예측값: 8 / 실제값: 